In [ ]:
URL = ''

In [1]:
import pandas as pd
from web3 import Web3

w3 = Web3(Web3.HTTPProvider(URL))
w3.isConnected()

True

In [2]:
import datetime
datetime.datetime.utcfromtimestamp(0)

datetime.datetime(1970, 1, 1, 0, 0)

In [3]:
df = pd.read_csv('../resources/ethereum_txs.csv').head()

In [4]:
def tx_receipt(tx_hash):
    return w3.eth.get_transaction_receipt(tx_hash)

def execution_timestamp_from_tx_hash(receipt):
    ts = w3.eth.get_block(receipt.get('blockNumber')).timestamp
    return pd.to_datetime(ts,unit='s')

def gas_cost(receipt):
    return receipt.get('gasUsed')
    
df['receipt'] = df['hash'].apply(lambda x: tx_receipt(x))

### 1. Assuming block length is 13 seconds, compute the approximate execution timestamp of each transaction.

In [5]:
df['approx_exec_timestamp'] = df['receipt'].apply(lambda x: execution_timestamp_from_tx_hash(x))

### 2. Compute the gas cost of each transaction in Gwei (1e-9 ETH).

In [6]:
gwei_to_eth = 0.000000001
df['gas_cost_gwei'] = df['receipt'].apply(lambda x: gas_cost(x))
df['gas_cost_eth'] = df['receipt'].apply(lambda x: gas_cost(x) * gwei_to_eth)

### 3. Using Coingecko's API, retrieve the approximate price of ETH at transaction execution time and compute the dollar cost of gas used.

In [7]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [8]:
def eth_hist_price(date):
    eth_hist_price = cg.get_coin_history_by_id(id='ethereum',date=date, localization='false')
    return eth_hist_price["market_data"]["current_price"]["usd"]

df['eth_to_usd'] = df['approx_exec_timestamp'].apply(lambda x: eth_hist_price(x.strftime('%d-%m-%Y')))
df['gas_cost_usd'] = df['gas_cost_eth'] * df['eth_to_usd']

In [10]:
df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,receipt,approx_exec_timestamp,gas_cost_gwei,gas_cost_eth,eth_to_usd,gas_cost_usd
0,0x0ba5abf4ef9eedb75a7fd5e645034288e02c6c4fafeb...,4,194,0x0057be07beef5d9b4beb9e2d147906e83d1915c8,0x8f26d7bab7a73309141a291525c965ecdea7bf42,0,287272,32035059237,0x51dbb2a7000000000000000000000000000000000000...,13012784,...,32035059237,32035059237,2,32035059237,"(blockHash, blockNumber, contractAddress, cumu...",2022-07-04 18:40:00,186594,0.000187,1074.607318,0.200515
1,0xd9f7b8f96374bfc016b7806e6009d88bd3471cebdeb5...,64,32,0x05db73fa65d49f58aab135a2cb00ecb323d37734,0xa906eef381cf828f0f2bfcc6e714a34fd4e80399,0,176541,35887544445,0xa0712d68000000000000000000000000000000000000...,2344231,...,315000000000,5250000000,2,35887544445,"(blockHash, blockNumber, contractAddress, cumu...",2022-07-04 18:40:00,28547,0.000029,1074.607318,0.030677
2,0xe27d930e2abe7de368f0c7189dce87bb10b418243f04...,69,189,0x28ac4256cf83d83a299547398c2ad40e17d641ee,0xf9936bc175f4777d85351738ef869b3ad7e7072d,0,120510,32047544445,0x5915d806000000000000000000000000000000000000...,12593820,...,33997117753,1410000000,2,32047544445,"(blockHash, blockNumber, contractAddress, cumu...",2022-07-04 18:40:00,96408,0.000096,1074.607318,0.103601
3,0x44eeacb135324e9195da609a7984df7bd8264e453bca...,315,7,0x1b915e753b3e213144092b0235910d35fc3288fb,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,50000000000000000,216629,31000000000,0xb6f9de95000000000000000000000000000000000000...,599785,...,31000000000,31000000000,2,31000000000,"(blockHash, blockNumber, contractAddress, cumu...",2022-07-04 20:56:32,145659,0.000146,1074.607318,0.156526
4,0x3ea88f5873d71c5534048b9444e0f0c76c2e78d30ec8...,6,126,0x6746222866a97b72c97d30e47c3783c68c762ed2,0x283af0b28c62c092c9727f1ee09c02ca627eb7f5,0,46267,17491940126,0xf14fcbc853f93a8e9bb207a3157b27ac5bb3da037807...,14320103,...,19239188764,1243765450,2,17491940126,"(blockHash, blockNumber, contractAddress, cumu...",2022-07-04 20:56:32,26170,0.000026,1074.607318,0.028122
